In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from datetime import datetime

import requests
from tqdm import tqdm

In [ ]:
def get_energy_data():
    # get all available time stamps
    stampsurl = "https://www.smard.de/app/chart_data/410/DE/index_hour.json"
    response = requests.get(stampsurl)
    # ignore first 6 years (don't need those in the baseline and speeds the code up a bit)
    timestamps = list(response.json()["timestamps"])[6 * 52 :]

    col_names = ["date_time", "Netzlast_Gesamt"]
    energydata = pd.DataFrame(columns=col_names)

    # loop over all available timestamps
    for stamp in tqdm(timestamps):
        dataurl = (
            "https://www.smard.de/app/chart_data/410/DE/410_DE_hour_"
            + str(stamp)
            + ".json"
        )
        response = requests.get(dataurl)
        rawdata = response.json()["series"]
        for i in range(len(rawdata)):
            rawdata[i][0] = datetime.fromtimestamp(
                int(str(rawdata[i][0])[:10])
            ).strftime("%Y-%m-%d %H:%M:%S")

        energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])

    energydata = energydata.dropna()
    energydata["date_time"] = pd.to_datetime(energydata.date_time) + pd.DateOffset(
        hours=1
    )  # adjust for correct time 'label'
    # set date_time as index
    energydata.set_index("date_time", inplace=True)

    return energydata

In [ ]:
df = get_energy_data()

In [ ]:
df.head()

Rename column for convenience

In [ ]:
df = df.rename(columns={"Netzlast_Gesamt": "gesamt"})

Rescale Netzlast so it fits requirements

In [ ]:
df["gesamt"] = df["gesamt"] / 1000

Check dtypes and if columns contain and missing values

In [ ]:
df.dtypes

In [ ]:
df.isna().any()

Define weekday column

In [ ]:
df["weekday"] = df.index.weekday  # Monday=0, Sunday=6
# df["time"] = df.index.strftime("%H:%M")

Lead times are

In [ ]:
horizons_def = [36, 40, 44, 60, 64, 68]  # [24 + 12*i for i in range(5)]
horizons_def

Adapt horzions so they actually fit

In [ ]:
horizons = [h + 0 for h in horizons_def]
horizons

In [ ]:
def get_date_from_horizon(last_ts, horizon):
    return last_ts + pd.DateOffset(hours=horizon)

In [ ]:
LAST_IDX = -1
LAST_DATE = df.iloc[LAST_IDX].name

In [ ]:
LAST_DATE

Get time and date that correspond to the lead times (starting at the last observation in our data which should be the respective thursday 0:00)  
*Attention*: if the last timestamp in the data is not thursday 0:00, you have to adjust your lead times accordingly

In [ ]:
horizon_date = [get_date_from_horizon(LAST_DATE, h) for h in horizons]
horizon_date

quantile levels

In [ ]:
tau = [0.025, 0.25, 0.5, 0.75, 0.975]

In [ ]:
# rows correspond to horizon, columns to quantile level
pred_baseline = np.zeros((6, 5))

In [ ]:
last_t = 100

for i, d in enumerate(horizon_date):
    weekday = d.weekday()
    hour = d.hour

    df_tmp = df.iloc[:LAST_IDX]

    cond = (df_tmp.weekday == weekday) & (df_tmp.index.time == d.time())

    pred_baseline[i, :] = np.quantile(df_tmp[cond].iloc[-last_t:]["gesamt"], q=tau)

In [ ]:
pred_baseline

Visually check if quantiles make sense

In [ ]:
x = horizons
_ = plt.plot(x, pred_baseline, ls="", marker="o", c="black")
_ = plt.xticks(x, x)
_ = plt.plot((x, x), (pred_baseline[:, 0], pred_baseline[:, -1]), c="black")

In [ ]:
from datetime import date

date_str = datetime.today().strftime("%Y%m%d")

In [ ]:
date_str = date.today()  # - timedelta(days=1)
date_str = date_str.strftime("%Y%m%d")
date_str

In [ ]:
df_sub = pd.DataFrame(
    {
        "forecast_date": date_str,
        "target": "energy",
        "horizon": [str(h) + " hour" for h in horizons_def],
        "q0.025": pred_baseline[:, 0],
        "q0.25": pred_baseline[:, 1],
        "q0.5": pred_baseline[:, 2],
        "q0.75": pred_baseline[:, 3],
        "q0.975": pred_baseline[:, 4],
    }
)
df_sub

In [ ]:
# need to change this
PATH = "/save/to/path"


df_sub.to_csv(PATH + date_str + "_power_benchmark.csv", index=False)